# Code organisation 

As I'm responsible for the output from more and more programmers I thought it was about time to write down some notes on good habits for organising code.

Perhaps this notebook will evolve from a remember-list - a kind of template for a todo-list - to a verification scripts. I'm not sure, but the prospects is there.


To keep the realism high I base the notes on the [tablite](https://github.com/root-11/tablite) project, instead of some theoretical `myproject` as that would feel artificial.


## Laying the foundations.

[1] Get the __right__ python version installed!

[2] Create virtual environment with 

```c:\....\Python311\Scripts\python.exe -m venv d:\venvs\tablite311``` 

I prefer calling the venv something meaningful so there's an association between project and python version - hence the 311 postfix.

[3] Activate the virtual environment: `d:\venvs\tablite\Scripts\activate.bat`. You can see that the venv is "active" as the commandline has the bracketed venv prefix:

```
(myproject) d:\
```

[4] Install pytests: `(tablite311) d:\github\tablite> pip install pytest`


[5] Run pytests on the empty project: `(tablite) d:\github\tablite> pytest` to see that there are no errors and that your python version reveals that you're the right virtual environment.

[6] create the project folder `d:\github\tablite` with the following contents. We will add content later, but if you're burning to see details, go to the [tablite source repo](https://github.com/root-11/tablite) and look at the files:

```
.github\workflows
    python-package.yml
.vscode\
    launch.json
    settings.json
dist\
tablite\
    __init__.py
    core.py  # sometimes called src.py or main.py
    utils.py
tests\
    data\
    __init__.py
    test_basics.py
.gitignore
LICENSE
README.md
setup.py
tutorial.ipynb
```

Notice that only the [README.md](https://github.com/root-11/tablite/blob/master/README.md) and [LICENSE](https://github.com/root-11/tablite/blob/master/LICENSE) are in uppercase. The PEP-8 (guideline) recommends that files and functions are in the pot_hole case, and only Classes are in the CamelCase, so let's stick to that.

You may think that `LICENSE` is not important in your project. Good faith or naivety however will not be a legal cover in every country. And as your code is on the **world wide** web, you could face charges if you land in a country where someone wants to blame you for neglect. So just add that [MIT license](https://opensource.org/licenses/MIT) and move on.

The folder `dist\` is deliberately empty. It will contain the packages that you'll upload to pypi when you create them (later), so just leave it for now.

We will need to commit all of this to version control. For this to be a breeze we would like a `.gitignore` file with this content:

```
# IDE configs.
.idea/   
*.code-workspace
.vscode/

# Compiled files
*.pyc

# Notebook checkpoints
.ipynb_checkpoints/

# Packages
*.egg-info
dist/
```

The `.gitignore` files means that we have a clean view of what is monitored by version control. 

Using `#` for comments on what those different escapes mean is a great help. Use it!


The `tests\` are deliberately kept seperate from the `tablite\` so that `tests\` can import the modules from `tablite` just as if they were installed using pip.


## Make the architecture work as the first thing.

There is nothing worse than spending time to get a project to run it's own test suite. 
For VScode this is simplified using the contents of the `.vscode` folder:

`launch.json` contains this:
```
{
    // Use IntelliSense to learn about possible attributes.
    // Hover to view descriptions of existing attributes.
    // For more information, visit: https://go.microsoft.com/fwlink/?linkid=830387
    "version": "0.2.0",
    "configurations": [
        {
            "name": "Python: Current File (Integrated Terminal)",
            "type": "python",
            "request": "launch",
            "program": "${file}",
            "console": "InternalConsole",
            "redirectOutput": true
        }
    ]
}
```
which instructs VScode to run any active file you're using.

You'll also need to send pytest the `-sv` arguments to assure that you can see any printouts as pytest otherwise hides them. This is set in `settings.json`:

```
{
    "python.defaultInterpreterPath": "C:/Data/venv/tablite310/Scripts/python.exe",
    "python.testing.pytestArgs": [
        "tests",
        "-sv"
    ],
    "python.testing.unittestEnabled": false,
    "python.testing.pytestEnabled": true

    
}
```


# Let's commit all of this to git.

Next install `git` and set your credentials: 
```
git config --global user.name “[firstname lastname]”
git config --global user.email “[valid-email]”
```

Now you're ready to use git. read the [cheat sheet from github](https://education.github.com/git-cheat-sheet-education.pdf) and commit to version control by:
```
d:\github\tablite> git init
d:\github\tablite> git commit -a "initial commit"
```



## Imports

Imagine we have a module called `x.py`

```# x.py
def dumps(text):
    return bytes(text)
```

and you choose to type

```
from x import *
from json import dumps
```
What exactly is going to happen in your namespace?

1. First `dumps` is imported from `x`.
2. Then `dumps` is imported from `json`, which overrides the variable declaration from `x`.

**Conclusion**

Do never ever use `from x import *`, as you will have no idea what exists in the namespace.






# Logging

Logging is supposed to be simple. Let's keep it that way.

In your module start with importing pythons logging module

```
import logging

logger = logging.getLogger()

try:
    logger.debug("this works?")
    # your code here
    logger.debug("it worked! ")
except Exception:
    logger.debug("it didn't!" )
    raise

```

In your test, you can set the log level to figure out what might be wrong:

```
import logging
logging.setLevel(logging.DEBUG)

# your code goes here.

```
As your test progresses, you'll see all the log messages.





# Checking packages

So you've built a nice package and uploaded it to pypi. That's nice. Does it work? - Let's check that other users can use it.

First let's create a empty / fresh virtual environment using the original python:

```
C:\Users\madsenbj\AppData\Local\Programs\Python\Python311\python.exe -m venv c:\Data\venv\tmp
```

Next we activate that environment

```
c:\Data\venv\tmp\Scripts\activate.bat
```

Then we do a test-install straight from pypi

```
(tmp) D:> pip install myproject
```

If you get an error like this, don't panic. The hint for the error is in line 3:

```
PS D:\gitlab\mfdesigner> & C:/Data/venv/mfdesigner310/Scripts/Activate.ps1
(mfdesigner310) PS D:\gitlab\mfdesigner> pip install tablite==2022.7.dev5
Collecting tablite==2022.7.dev5
  Using cached tablite-2022.7.dev5.tar.gz (51 kB)  <---------------- HINT: cached!
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.

<cut for brevity>

× Encountered error while generating package metadata.
╰─> See above for output.
```

You've probably built a package earlier and now as you use `pip install` pip uses the cached version. NOT the version that you've just uploaded.

To overcome this problem, make it a habit to use `pip install myproject --no-cache` and now it will work:


```
(mfdesigner310) PS D:\gitlab\mfdesigner> pip install tablite==2022.7.dev5 --no-cache
Collecting tablite==2022.7.dev5
  Downloading tablite-2022.7.dev5-py3-none-any.whl (54 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 1.4 MB/s eta 0:00:00

<cut for brevity>

Collecting et-xmlfile
  Downloading et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
Using legacy 'setup.py install' for mplite, since package 'wheel' is not installed.
Using legacy 'setup.py install' for pyperclip, since package 'wheel' is not installed.
Building wheels for collected packages: graph-theory
  Building wheel for graph-theory (pyproject.toml) ... done
  Created wheel for graph-theory: filename=graph_theory-2022.3.9.54615-py3-none-any.whl size=53795 sha256=d675a72c18d0ebf704e737efbd135f80eaac4c12eb65d5f0752838418863bdde
  Stored in directory: C:\Users\madsenbj\AppData\Local\Temp\pip-ephem-wheel-cache-k2ntov1y\wheels\5c\01\5b\8f28bd95cf08edb938b43af28997be35b9e51dc7586036705a
Successfully built graph-theory
Installing collected packages: xlwt, texttable, pyuca, pyperclip, lml, graph-theory, xlrd, pyexcel-io, psutil, numpy, lxml, et-xmlfile, colorama, tqdm, pyexcel-xls, pyexcel-odsr, pyexcel, openpyxl, h5py, pyexcel-xlsx, mplite, tablite
  Running setup.py install for pyperclip ... done
  Running setup.py install for mplite ... done
Successfully installed colorama-0.4.5 et-xmlfile-1.1.0 graph-theory-2022.3.9.54615 h5py-3.7.0 lml-0.1.0 lxml-4.9.1 mplite-1.1.0 numpy-1.23.1 openpyxl-3.0.10 psutil-5.9.1 pyexcel-0.7.0 pyexcel-io-0.6.6 pyexcel-odsr-0.6.0 pyexcel-xls-0.7.0 pyexcel-xlsx-0.6.0 pyperclip-1.8.2 pyuca-1.2 tablite-2022.7.dev5 texttable-1.6.4 tqdm-4.64.0 xlrd-2.0.1 xlwt-1.3.0
(mfdesigner310) PS D:\gitlab\mfdesigner> 

```

Now we're good. We __know__ it works.








